In [7]:
import os
os.chdir("F:\mlproject1\mlproject")

In [8]:
%pwd

'F:\\mlproject1\\mlproject'

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [35]:
%pwd

'f:\\mlproject1'

In [2]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [3]:
from pathlib import Path
class ConfigurationManager:
    def __init__(
        self, 
        config_path = config_path,
        params_path = params_path,
        schema_path = schema_path
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path) 
        
        create_directories([self.config.artifacts_root])   
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),  # Convert to Path
            unzip_dir=Path(config.unzip_dir)  # Convert to Path
        )

        return data_ingestion_config



In [51]:
import os
import zipfile
import urllib.request
from pathlib import Path
from mlproject import logger
from mlproject.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the data file if it does not already exist locally."""
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        Function returns None.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  # Ensure the directory exists

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
config_manager = ConfigurationManager()
data_ingestion_config = config_manager.get_data_ingestion_config()
data_ingestion = DataIngestion(data_ingestion_config)
data_ingestion.download_data()
data_ingestion.extract_zip_file()


[2025-02-11 16:59:08,337: INFO: common: Created directory at: artifacts]
[2025-02-11 16:59:08,337: INFO: common: Created directory at: artifacts/data_ingestion]
Downloaded artifacts\data_ingestion\data.zip successfully.
Data already extracted at artifacts\data_ingestion, skipping extraction.
